<a href="https://colab.research.google.com/github/Vigrahalabhanu3/FFML-226Q1A4409/blob/main/module_01_lab_02_mlpractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


In [4]:
type(dataset), dataset.DESCR

(sklearn.utils._bunch.Bunch,
 '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n--------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 20640\n\n    :Number of Attributes: 8 numeric, predictive attributes and the target\n\n    :Attribute Information:\n        - MedInc        median income in block group\n        - HouseAge      median house age in block group\n        - AveRooms      average number of rooms per household\n        - AveBedrms     average number of bedrooms per household\n        - Population    block group population\n        - AveOccup      average number of household members\n        - Latitude      block group latitude\n        - Longitude     block group longitude\n\n    :Missing Attribute Values: None\n\nThis dataset was obtained from the StatLib repository.\nhttps://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html\n\nThe target variable is the median house value for California districts,\nexpressed in hundreds of 

Here is a function for calculating the 1-nearest neighbours

In [5]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [6]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [7]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [8]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [9]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [10]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [11]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [12]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [13]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [14]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Answer for first question
Increase Percentage of Validation Set:

Pros:
Increased Generalization: With a larger validation set, you are providing the model with more diverse data to evaluate its performance. This can lead to a better estimate of how well your model generalizes to unseen data.
Reduced Overfitting: A larger validation set may make it more difficult for the model to overfit because it has less training data available.
Cons:
Reduced Training Data: As you increase the size of the validation set,you have less data available for training the Decrease Percentage of Validation Set:

Pros:
More Training Data: With a smaller validation set, you have more data available for training the model. This can lead to better performance on the training data.
Faster Training: Training may be faster because the model spends less time evaluating its performance on the smaller validation set.
Cons:
Risk of Overfitting: A smaller validation set may make it easier for the model to overfit because it has less data to evaluate its performance model. This can result in a model that doesn't perform as well on the training data.
Slower Training: Training may take longer because the model has to evaluate its performance on a larger validation set after each training iteration.nd prevent overfitting.
Less Reliable Evaluation: The evaluation of the model's performance on a smaller validation set might be less reliable and may not provide a good estimate of how well the model will generalize to new, unseen data.
The choice of the percentage of the validation set depends on the specific problem, the size of your dataset, and the trade-off between training data and reliable model evaluation. In practice, a common split is 70-80% for training and 20-30% for validation, but these percentages can vary based on the circumstances. It's important to strike a balance that allows your model to learn effectively from the data while still providing a robust evaluation of its performance. Techniques like cross-validation can also be used to mitigate the effects of varying validation set sizes.

2.Answer for second question
Data Representativeness:

If the validation set is too small or unrepresentative of the data distribution, the accuracy on the validation set may not accurately reflect how well the model will perform on the test set. This can result in an optimistic or pessimistic estimate of model performance.
Overfitting:

If the training set is much larger than the validation set, the model may have a higher risk of overfitting to the training data. In this case, the model could perform exceptionally well on the training set but poorly on the validation and test sets, leading to a mismatch between validation and test set accuracy.
Underfitting:

Conversely, if the validation set is significantly larger than the training set, the model may not be able to learn a sufficiently complex representation of the data during training, potentially leading to underfitting. This can result in lower validation and test set accuracy.
Model Complexity:

The relationship between train, validation, and test set sizes also depends on the complexity of the model being trained. More complex models may require larger training datasets to generalize well. If the validation set is too small, it may not provide an accurate assessment of the model's ability to generalize.
Statistical Variability:

Smaller datasets, in general, tend to exhibit more variability in performance metrics. A larger validation set can help reduce this variability and provide a more stable estimate of model performance.
To ensure a reliable prediction of test set accuracy using the validation set, it's important to strike a balance between the sizes of the training and validation sets and ensure they are representative of the overall dataset distribution. Cross-validation techniques, such as k-fold cross-validation, can help mitigate some of the issues associated with varying set sizes by repeatedly splitting the data into training and validation subsets and averaging the results.

Ultimately, the key is to select the sizes of the training and validation sets based on the specific problem, dataset size, and the complexity of the model to achieve a robust estimate of the model's generalization performance on unseen data (the test set).



3.Answer for third question


Rule of thumb:A common practice is to split your data into three sets:training,validation,and test.Common split ratios are 70-15-15,80-10-10,or 60-20-20,where the first number represents the percentage allocated to training and validation, while still reserving a separate test set for final evaluation.

Size of Dataset: if you have a small dataset, you might lean towards a larger validation set because you have limited data for training.For very large datasets,a smaller validation set may be sufficient.

Complexity of Models:if you're working with a complex model(e.g.,deep neural networks), you may need a larger validation set to detect overfitting.Simpler models may require a smaller validation set.

Double-click(or enter)to edit

Double-click(or enter)to edit

Double-click(or enter)to edit

Double-click(or enter)to edit













## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [15]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [16]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Answer for first question:

educed Variance: By averaging the results of multiple runs on different data splits, cross-validation helps reduce the impact of random variability in the data. This means that the evaluation metric (such as accuracy) becomes more stable and representative of the model's true performance.

Better Generalization: Cross-validation provides a more comprehensive view of how well your model generalizes to different data partitions. This is important for assessing whether your model is overfitting or underfitting the training data.

Utilizes the Entire Dataset: Cross-validation allows you to make efficient use of your entire dataset for both training and validation, as each data point is used in the validation set exactly once across the k runs.

Robustness to Data Imbalances: In situations where your dataset has imbalanced classes, cross-validation helps ensure that each class is represented in the validation set, reducing the risk of biased evaluation.

Model Selection: Cross-validation can also be used for hyperparameter tuning and model selection, as it provides a more robust estimate of how different model configurations perform on unseen data.


While cross-validation is a valuable technique for assessing model performance, keep in mind that it can be computationally expensive, especially if your dataset is large or if training a single model takes a long time. In such cases, you may consider alternative methods like stratified sampling or hold-out validation, depending on your specific needs and constraints.


Answer for second question:

Data Quality: High-quality training and test data are crucial for accurate estimates. If the data used to train and test the model is representative of the real-world distribution, the estimates are likely to be more accurate.

Model Complexity: The complexity of the model can impact accuracy estimates. A more complex model might overfit the training data and provide an overly optimistic estimate of test accuracy. On the other hand, a simple model might underfit and provide a pessimistic estimate. Techniques like cross-validation can help mitigate this issue.

Regularization: Regularization techniques (e.g., L1, L2 regularization) can help prevent overfitting, leading to more accurate estimates of test accuracy.

Cross-Validation: Cross-validation involves splitting the data into multiple subsets and training/testing the model on different combinations of these subsets. It provides a more robust estimate of model performance by reducing the impact of the initial data split.

Hyperparameter Tuning: Properly tuned hyperparameters, such as learning rates and batch sizes, can lead to a more accurate estimate of test accuracy.

Ensemble Methods: Ensemble methods, like bagging and boosting, can combine multiple models to improve accuracy estimates.

Evaluation Metrics: The choice of evaluation metrics can also impact the perceived accuracy of a model. For example, accuracy might not be the best metric for imbalanced datasets; precision, recall, F1-score, or AUC might provide a more accurate assessment.

Bias and Fairness: Assessing test accuracy should also consider potential biases and fairness issues in the data and model. Accuracy alone may not capture these issues accurately.

External Validation: Testing a model on external datasets that were not used during training can provide a more accurate estimate of its generalization performance.

Domain-Specific Considerations: The domain and application of the model can greatly affect what constitutes an accurate estimate. Some applications may require specialized evaluation techniques.

In summary, whether a model provides a more accurate estimate of test accuracy depends on various factors, including data quality, model complexity, regularization, evaluation techniques, and more. Researchers and practitioners often use a combination of these approaches to obtain the most reliable and accurate estimates of a model's performance on unseen data.


3.Answer for third question:

Overfitting: When you have a very small training dataset, increasing the number of iterations can lead to overfitting. Overfitting occurs when the model learns to fit the noise in the training data rather than the underlying patterns. This can result in poor generalization to new, unseen data.

Lack of Diversity: Small datasets may not capture the full diversity of the underlying data distribution. Increasing iterations won't necessarily help if the training data doesn't represent the true complexity of the problem.

Computational Costs: Training a model for a large number of iterations can be computationally expensive, especially for deep neural networks. It may not be practical or efficient to use a large number of iterations on a small dataset.

Hyperparameter Tuning: In addition to increasing iterations, you may need to carefully tune other hyperparameters like learning rate, batch size, and regularization to mitigate overfitting and achieve good results on a small dataset.

Data Augmentation: To make the most of a small dataset, you can also employ data augmentation techniques to artificially increase the effective size of your training dataset. This involves generating new training examples by applying various transformations (e.g., rotation, translation, cropping) to the existing data.

Transfer Learning: Another approach for small datasets is to use pre-trained models and fine-tune them on your specific task. Transfer learning allows you to leverage knowledge from models trained on large datasets and adapt them to your smaller dataset.

In summary, while increasing the number of iterations can help a model learn from a small dataset to some extent, it's not a magic solution and should be done carefully to avoid overfitting. You may need to combine it with other techniques like data augmentation, hyperparameter tuning, and transfer learning to achieve good performance on a limited amount of data.


4.answer for fourth question:


Overfitting: When you have a small dataset and you train for a large number of iterations, there's a higher risk of overfitting. Overfitting occurs when your model learns to memorize the training data rather than generalize from it. This can lead to poor performance on unseen data.

Computational Resources: Increasing the number of iterations may require more computational resources, including time and hardware. Training deep learning models for a large number of iterations can be computationally expensive.

Early Stopping: To combat overfitting, it's important to use techniques like early stopping. Early stopping involves monitoring the model's performance on a validation dataset during training and stopping training when the performance starts to degrade. This prevents the model from continuing to learn noise in the data.

Data Augmentation: Another strategy to address a small dataset is data augmentation. This involves creating new training examples by applying random transformations to the existing data (e.g., rotating, flipping, or cropping images). Data augmentation can effectively increase the effective size of your training dataset.

Transfer Learning: If you have access to a pretrained model on a related task with a larger dataset, transfer learning can be a powerful technique. You can fine-tune the pretrained model on your smaller dataset, which often requires fewer iterations and can yield good results.

Regularization: Applying regularization techniques such as dropout or L1/L2 regularization can help prevent overfitting, even with a small dataset.

In summary, while increasing the number of iterations can help to some extent when dealing with a small dataset, it should be done cautiously and in conjunction with other strategies like data augmentation, regularization, and early stopping. The best approach may vary depending on the specific problem and dataset you are working with.


















